In [1]:
import numpy as np
import torch

In [2]:
a = [1., 2., 3.]

In [3]:
a[0]

1.0

In [4]:
a[2] = 3.
a

[1.0, 2.0, 3.0]

In [5]:
a = torch.ones(3)

In [6]:
a

tensor([1., 1., 1.])

In [7]:
float(a[1])

1.0

In [8]:
a[2] = 2.

In [9]:
a

tensor([1., 1., 2.])

In [10]:
points = torch.zeros(6)
points[0] = 4.
points[1] = 1.
points[2] = 5.
points[3] = 3.
points[4] = 2.
points[5] = 1.

In [11]:
points

tensor([4., 1., 5., 3., 2., 1.])

In [12]:
points = torch.tensor([4., 1., 5., 3., 2., 1.])

In [13]:
points

tensor([4., 1., 5., 3., 2., 1.])

In [14]:
float(points[0]), float(points[1])

(4.0, 1.0)

In [15]:
points = torch.tensor([[4., 1.], [5., 3.], [2., 1.]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [16]:
points.shape

torch.Size([3, 2])

In [17]:
points = torch.zeros(3, 2)
points

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [18]:
points = torch.tensor([[4., 1.], [5., 3.], [2., 1.]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [19]:
points[0, 1]

tensor(1.)

In [20]:
points[0][1]

tensor(1.)

In [21]:
points[0]

tensor([4., 1.])

In [22]:
some_list = list(range(6))
some_list[:]

[0, 1, 2, 3, 4, 5]

In [23]:
some_list[1:4]

[1, 2, 3]

In [24]:
some_list[1:]

[1, 2, 3, 4, 5]

In [25]:
some_list[:4]

[0, 1, 2, 3]

In [26]:
some_list[:-1]

[0, 1, 2, 3, 4]

In [27]:
some_list[1:4:2]

[1, 3]

In [28]:
points[None]

tensor([[[4., 1.],
         [5., 3.],
         [2., 1.]]])

In [29]:
np_points = np.array([[4., 1,], [5., 3.], [2., 1.]])

In [30]:
np_points[None]

array([[[4., 1.],
        [5., 3.],
        [2., 1.]]])

In [31]:
from PIL import Image
from torchvision import transforms

preproscess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

img = Image.open("../../data/p1ch2/bobby.jpg")
img_t = preproscess(img)

In [32]:
img_t = torch.randn(3, 5, 5)
weight = torch.tensor([0.2126, 0.7152, 0.0722])

In [33]:
batch_t = torch.randn(2, 3, 5, 5)

In [34]:
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch_t.mean(-3)
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [35]:
test_mean = np.arange(1, 25).reshape(2, 3, 4)
test_mean

array([[[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12]],

       [[13, 14, 15, 16],
        [17, 18, 19, 20],
        [21, 22, 23, 24]]])

In [36]:
test_mean.mean(-1)

array([[ 2.5,  6.5, 10.5],
       [14.5, 18.5, 22.5]])

In [55]:
unsqueezed_weights = weight.unsqueeze(-1).unsqueeze_(-1)
img_weights = (img_t * unsqueezed_weights)
batch_weights = (batch_t * unsqueezed_weights)
img_gray_weighted = img_weights.sum(-3)
batch_gray_wighted = batch_weights.sum(-3)
batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

In [38]:
img_gray_weighted_fancy = torch.einsum('...chw,c->...hw', img_t, weight)
batch_gray_wighted_fancy = torch.einsum('...chw,c->...hw', batch_t, weight)
batch_gray_wighted_fancy.shape

torch.Size([2, 5, 5])

In [39]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])
weights_named

C:\Users\khjop\AppData\Local\Temp\ipykernel_11720\2338252810.py:1: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10/core/TensorImpl.h:1408.)
  weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])


tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [40]:
img_named = img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print('img named: ', img_named.shape, img_named.names)
print('batch named: ', batch_named.shape, batch_named.names)

img named:  torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named:  torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [41]:
weights_aligned = weights_named.align_as(img_named)
weights_aligned.shape, weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

In [42]:
gray_named = (img_named * weights_aligned).sum('channels')
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

In [43]:
gray_plain = gray_named.rename(None)
gray_plain.shape, gray_plain.names

(torch.Size([5, 5]), (None, None))

In [44]:
double_points = torch.ones(10, 2, dtype=torch.double)
short_points = torch.tensor([[1, 2], [3, 4]], dtype=torch.short)

In [45]:
double_points.dtype, short_points.dtype

(torch.float64, torch.int16)

In [46]:
double_points = torch.zeros(10, 2).double()
short_points = torch.ones(10, 2).short()

In [47]:
double_points.dtype, short_points.dtype

(torch.float64, torch.int16)

In [48]:
a = torch.ones(3, 2)
a_t = torch.transpose(a, 0, 1)

In [49]:
a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

In [50]:
a = torch.ones(3, 2)
a_t = a.transpose(0, 1)

In [51]:
a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

In [52]:
points = torch.tensor([[4., 1.], [5., 3.], [2., 1.]])
points.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage._TypedStorage(dtype=torch.float32, device=cpu) of size 6]

In [53]:
points_storage = points.storage()
points_storage[0]

4.0

In [54]:
points.storage()[1]

1.0

In [56]:
points = torch.tensor([[4., 1.], [5., 3.], [2., 1.]])
points_storage = points.storage()
points_storage[0] = 2.
points

tensor([[2., 1.],
        [5., 3.],
        [2., 1.]])

In [57]:
a = torch.ones(3, 2)

In [58]:
a

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])

In [59]:
a.zero_()

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [60]:
a

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [61]:
points = torch.tensor(([[4., 1.], [5., 3.], [2., 1.]]))
second_point = points[1]

In [62]:
second_point

tensor([5., 3.])

In [63]:
second_point.storage_offset()

2

In [64]:
second_point.size()

torch.Size([2])

In [66]:
second_point.shape

torch.Size([2])

In [67]:
second_point = points[1]
second_point.size()

torch.Size([2])

In [68]:
points = ([[4., 1.], [5., 3.], [2., 1.]])
second_point = points[1]
second_point[0] = 10.
points

[[4.0, 1.0], [10.0, 3.0], [2.0, 1.0]]

In [69]:
points = torch.tensor([[4., 1.], [5., 3.], [2., 1.]])
second_point = points[1].clone()
second_point[0] = 10
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [70]:
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [71]:
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [73]:
id(points.storage()) == id(points_t.storage())

True

In [74]:
points.stride()

(2, 1)

In [75]:
points_t.stride()

(1, 2)

In [78]:
some_t = torch.ones(3, 4, 5)
tranpose_t = some_t.transpose(0, 2)
some_t.shape

torch.Size([3, 4, 5])

In [79]:
tranpose_t.shape

torch.Size([5, 4, 3])

In [80]:
some_t.stride()

(20, 5, 1)

In [82]:
tranpose_t.stride()

(1, 5, 20)

In [85]:
points.is_contiguous()

True

In [86]:
points_t.is_contiguous()

False

In [87]:
points = torch.tensor([[4., 1.], [5., 3.], [2., 1.]])

In [88]:
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [89]:
points_t.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage._TypedStorage(dtype=torch.float32, device=cpu) of size 6]

In [90]:
points_t.stride()

(1, 2)

In [99]:
import imageio.v2 as imageio

In [100]:
img_arr = imageio.imread("../../data/p1ch4/image-dog/bobby.jpg")

In [101]:
img_arr.shape

(720, 1280, 3)

In [102]:
img = torch.from_numpy(img_arr)
out = img.permute(2, 0, 1)

In [104]:
batch_size = 3
batch = torch.zeros(batch_size, 3, 256, 256, dtype=torch.uint8)

In [105]:
import os

In [107]:
data_dir = "../../data/p1ch4/image-cats/"
filenames = [name for name in os.listdir(data_dir)
             if os.path.splitext(name)[-1] == '.png']
for i, filename in enumerate(filenames):
    img_arr = imageio.imread(os.path.join(data_dir, filename))
    img_t = torch.from_numpy(img_arr)
    img_t = img_t.permute(2, 0, 1)
    img_t = img_t[:3]
    batch[i] = img_t

In [108]:
batch = batch.float()
batch /= 255.0